<a href="https://colab.research.google.com/github/Kumar-Sanskar-2724/Vision-Transformer/blob/main/Vision_Transformer_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import torch
import torch.nn as nn
import torchvision
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets,transforms
import timm
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
from PIL import Image
# Get going_modular directory
try:
  from going_modular.going_modular import data_setup,engine
  from helper_functions import plot_loss_curves,set_seeds,download_data
except:
  print(f"[INFO] Couldn't find going_modular and helper_functions directory, downloading them from GitHub...")
  !git clone https://github.com/mrdbourke/pytorch-deep-learning
  !mv pytorch-deep-learning/going_modular .
  !mv pytorch-deep-learning/helper_functions.py .
  !rm -rf pytorch-deep-learning
  from going_modular.going_modular import data_setup,engine
  from helper_functions import download_data,set_seeds,plot_loss_curves

## Setting Devic Agnostic Code

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

In [ ]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))
])

## Getting Data

In [ ]:
train_dataset = datasets.CIFAR10(root='./data',train=True,transform=transform,download=True)
test_dataset = datasets.CIFAR10(root='./data',train=False,transform=transform,download=True)

In [ ]:
class_names=train_dataset.classes
class_names
class_idx = train_dataset.class_to_idx
class_idx

In [ ]:
example = download_data(source='https://github.com/Kumar-Sanskar-2724/Vision-Transformer/raw/refs/heads/main/examples/examples.zip',destination='examples')

In [ ]:
label_map = {
    0: 'airplane',
    1: 'automobile',
    2: 'bird',
    3: 'cat',
    4: 'deer',
    5: 'dog',
    6: 'frog',
    7: 'horse',
    8: 'ship',
    9: 'truck'
}

In [ ]:
train_dataloader = DataLoader(train_dataset,batch_size=64,shuffle=True)
test_dataloader = DataLoader(test_dataset,batch_size=64,shuffle=False)

## Setting up Model

In [ ]:
model = timm.create_model('vit_tiny_patch16_224',pretrained=True)
model.head = nn.Linear(in_features=model.head.in_features,out_features=10)
model.to(device)

## Creating a function to time our experiments

In [ ]:
from timeit import default_timer as timer
def print_train_time(start:float,
                     end:float,
                     device:torch.device=None):
  """ Prints difference between start time and end time"""
  train_time = end - start
  print(f"Total train time on:{device} {train_time:.3f} seconds")
  return train_time

## Training and Evaluation

In [ ]:
loss_fn=nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model.parameters(),lr=3e-5)

In [ ]:
from going_modular.going_modular import engine
set_seeds()
model_results = engine.train(model=model,
                             train_dataloader=train_dataloader,
                             test_dataloader=test_dataloader,
                             optimizer=optimizer,
                             loss_fn=loss_fn,
                             epochs=3,
                             device=device)

## Saving our model

In [ ]:
from going_modular.going_modular import utils
utils.save_model(model=model,
                 target_dir='models',
                 model_name='ViT_feature_extractor.pth')

## Model Deplyoment

In [ ]:
# Import/install gradio
try:
  import gradio as gr
except:
  !pip -q install gradio
  import gradio as gr

In [ ]:
# Auto-collect example images
import glob
example_list = [[path] for path in glob.glob("data/examples/*.jpg")]
example_list

In [ ]:
def predict(image):
    image = image.convert("RGB")
    input_tensor = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(input_tensor)
        predicted_class = output.argmax(1).item()

    return f"Predicted Class: {label_map[predicted_class]}"

In [ ]:
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs='text',
    examples= example_list,
    title="Vision Transformer CIFAR-10 Classifier",
    description="Upload a CIFAR-10 image, and the ViT Tiny model will predict the class."
)
demo.launch()

## Turning our CIFAR Model into a deployable app

### Creating a `demos` folder to store our CIFAR app files

In [ ]:
import pathlib
from pathlib import Path
import shutil

# Create CIFAR with demo path
cifar = Path('demo/cifar')

# Remove files that might exist and create a new directory
if cifar.exists():
  shutil.rmtree(cifar)
  cifar.mkdir(parents=True,exist_ok=True)
else:
  cifar.mkdir(parents=True,exist_ok=True)

!ls demo/cifar

## Creating a folder of example images to use with our CIFAR demo

In [ ]:
import shutil
from pathlib import Path

# Create an example directory
cifar_example_path = cifar/'examples'
cifar_example_path.mkdir(parents=True,exist_ok=True)

# Collect 3 random test dataset image path
cifar_examples = [Path('data/examples/Airplane.jpg'),
                            Path('data/examples/automobile.jpg'),
                            Path('data/examples/bird.jpg')]

# Copy the three images to the examples directory
for example in cifar_examples:
  destination = cifar_example_path/example.name
  print(f'[INFO] Copying {example} to {destination}')
  shutil.copy2(src=example,
              dst=destination)

In [ ]:
import os
example_list = [['examples/'+example] for example in os.listdir(cifar_example_path)]
example_list

## Moving our trained Tiny ViT model to our CIFAR demo directory

In [ ]:
import shutil

# Create a source path for our target model
tiny_vit_model_path = 'models/ViT_feature_extractor.pth'

# Create a destination path for our target model
tiny_vit_destination = cifar/tiny_vit_model_path.split('/')[1]

# Try to move the model file
try:
  print(f'[INFO] Attmepting to move :{tiny_vit_model_path} to {tiny_vit_destination}')

  # Move the model
  shutil.move(src=tiny_vit_model_path,
              dst=tiny_vit_destination)
  print(f'[INFO] Model move complete')

except:
  print(f"[INFO] No model found at {tiny_vit_model_path}, perhaps its already been moved?")
  print(f"[INFO] Model exists at {tiny_vit_destination}: {tiny_vit_destination.exists()}")

## Turning off Tiny_ViT model into a Python script (model.py)

In [ ]:
%%writefile demo/cifar/model.py
import torch
import torchvision
from torch import nn
import timm
from torchvision import transforms

def create_model(num_classes:int=10,
                 seeds:int=42):
  transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize((0.5,), (0.5,))])

  model = timm.create_model('vit_tiny_patch16_224',pretrained=True)
  model.head = nn.Linear(in_features=model.head.in_features,out_features=10)

  return model,transform

In [ ]:
class_names

## Turning our Tiny_ViT Gradio app into a Python script(app.py)

In [ ]:
%%writefile demo/cifar/app.py
### 1. Imports and class names setup ###
import gradio as gr
import os
import torch

from model import create_model
from timeit import default_timer as timer
from typing import Tuple,Dict

# Setup class names
class_names = ['airplane','automobile','bird','cat','deer','dog','frog','horse','ship','truck']

### 2. Model and transforms perparation ###
tiny_vit,tiny_vit_transforms = create_model(num_classes=10)

# Load save weights
tiny_vit.load_state_dict(torch.load(f='ViT_feature_extractor.pth',
                                    map_location=torch.device('cpu')))

# Predict function
def predict(image):
    # Make sure the image is in RGB
    image = image.convert("RGB")

    # Apply the necessary transformation for your model
    input_tensor = tiny_vit_transforms(image).unsqueeze(0)

    # Set model to evaluation mode
    tiny_vit.eval()
    with torch.no_grad():
        # Get the model's raw output (logits)
        output = tiny_vit(input_tensor)

        # Apply softmax to convert logits to probabilities
        pred_probs = torch.softmax(output, dim=1)

    # Create a dictionary mapping class names to their probabilities
    pred_label_and_probs = {
        class_names[i]: float(pred_probs[0][i]) for i in range(len(class_names))
    }

    return pred_label_and_probs

# Gradio app
title="Vision Transformer CIFAR-10 Classifier",
description="Upload a CIFAR-10 image, and the ViT Tiny model will predict the class."

# Creating example list
example_list =[['examples/'+example] for example in os.listdir('examples')]

# Create the gradio demo
demo = gr.Interface(
    fn=predict,
    inputs=gr.Image(type="pil"),
    outputs='text',
    examples= example_list,
    title="Vision Transformer CIFAR-10 Classifier",
    description="Upload a CIFAR-10 image, and the ViT Tiny model will predict the class."
)
demo.launch()

## Creating a requirement file `requirements.txt`

In [ ]:
%%writefile demo/cifar/requirements.txt
torch >= 1.12.0
torchvision >= 0.13.0
gradio >= 3.1.4
timm

##  Deploying our FoodVision Mini app HuggingFace Spaces

In [ ]:
!ls demo/cifar/examples/

In [ ]:
# Change into the foodvision_mini directory and then zip it from the inside
!cd demo/cifar && zip -r ../cifar.zip * -x "*.pyc" "*.ipynb" "*__pycache__*" "*ipynb_checkpoints*"

In [ ]:
# Download
try:
  from google.colab import files
  files.download('demo/cifar.zip')
except:
  print(f"Not running in Google Colab, can't use google.colab.files.download(), please download foodvision_mini.zip manually.")